In [2]:
from dotenv import load_dotenv
import os
from langchain_community.document_loaders import Docx2txtLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate

# Load environment variables
load_dotenv()


api_key = os.getenv("OPENAI_API_KEY")


# Document loading and processing
loader = Docx2txtLoader("./files/1984.docx")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
splits = text_splitter.split_documents(documents)

# Create vectorstore and retriever
embeddings = OpenAIEmbeddings(openai_api_key=api_key)  # API 키 직접 전달
vectorstore = FAISS.from_documents(splits, embeddings)
retriever = vectorstore.as_retriever()


# Initialize memory
memory = ConversationBufferMemory(
    memory_key="chat_history", return_messages=True, output_key="answer"
)

# Create custom prompt
prompt_template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know.

Context: {context}

Chat History: {chat_history}
Question: {question}
Answer: """

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "chat_history", "question"]
)

# Create the chain manually
chain = ConversationalRetrievalChain.from_llm(
    llm=ChatOpenAI(temperature=0),
    retriever=retriever,
    memory=memory,
    combine_docs_chain_kwargs={"prompt": PROMPT},
    return_source_documents=False,
    return_generated_question=False,
)

# Ask questions
questions = [
    "Is Aaronson guilty?",
    "What message did he write in the table?",
    "Who is Julia?",
]

for question in questions:
    result = chain.invoke({"question": question})
    print(f"\nQuestion: {question}")
    print(f"Answer: {result['answer']}")


Question: Is Aaronson guilty?
Answer: Yes, according to the context provided, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.

Question: What message did he write in the table?
Answer: Aaronson wrote "GOD IS POWER" on the table.

Question: Who is Julia?
Answer: Julia is a character mentioned in the context provided, who was involved in a relationship with the protagonist and was also subjected to torture and betrayal.
